In [ ]:
# import Pkg; Pkg.add("PlotlyJS")

In [25]:
include("astrobee_se3_script.jl")
using Plots


In [3]:
model = AstrobeeSE3()

AstrobeeSE3(13, 6, Any[], Any[], Any[], 1.0e-5, 7.2, [0.1083 0.0 0.0; 0.0 0.1083 0.0; 0.0 0.0 0.1083], [9.23361 0.0 0.0; 0.0 9.23361 0.0; 0.0 0.0 9.23361], [100.0, 100.0, 100.0, 0.4, 0.4, 0.4, 100.0, 100.0, 100.0, 100.0, 0.785, 0.785, 0.785], [-100.0, -100.0, -100.0, -0.4, -0.4, -0.4, -100.0, -100.0, -100.0, -100.0, -0.785, -0.785, -0.785], [0.72, 0.72, 0.72, 0.0944617, 0.0944617, 0.0944617], [-0.72, -0.72, -0.72, -0.0944617, -0.0944617, -0.0944617], [-0.25, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.7, -0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], 110.0, Any[Any[[0.4, -0.1, 0.0], 0.1]], Any[PolygonalObstacle([0.0, 0.2, 0.2], 0.05, 0.05, 0.05, Array{Int64,1}[[0, -1, 0], [1, 0, 0], [0, 1, 0], [-1, 0, 0], [0, 0, 1], [0, 0, -1]], Array{Float64,1}[[0.05, 0.15, 0.25], [0.05, 0.25, 0.25], [-0.05, 0.25, 0.25], [-0.05, 0.15, 0.25], [0.05, 0.15, 0.15], [0.05, 0.25, 0.15], [-0.05, 0.25, 0.15], [-0.05, 0.15, 0.15]])], 1000.0, 1.0, 1.0e9, 0.001, 0.01, 50.0, 2.0

instantiated a floating joint


UndefVarError: UndefVarError: TOS_SCP not defined

In [ ]:
# verts = [(-1.0, 1.0), (-1.28, 0.6), (-0.2, -1.4), (0.2, -1.4), (1.28, 0.6), (1.0, 1.0), (-1.0, 1.0), (-0.2, -0.6), (0.0, -0.2), (-0.4, 0.6), (1.28, 0.6), (0.2, -1.4), (-0.2, -1.4), (0.6, 0.2), (-0.2, 0.2), (0.0, -0.2), (0.2, 0.2), (-0.2, -0.6)]
# x = 0.1:0.2:0.9
# y = 0.7 * rand(5) .+ 0.15
# plot(x, y, line=(3, :dash, :lightblue), marker=(Shape(verts), 30, RGBA(0, 0, 0, 0.2)), bg=:pink, fg=:darkblue, xlim=(0, 1), ylim=(0, 1), leg=false)


In [ ]:
# verts = [(0.0, 0.0, 0.0), (0.0, 1.0, 0.0), (1.0, 1.0, 0.0),(1.0, 0.0, 0.0),
# (0.0, 0.0, 1.0), (0.0, 1.0, 1.0), (1.0, 1.0, 1.0),(1.0, 0.0, 1.0)]
# x = 0.1:0.2:0.9
# y = 0.7 * rand(5) .+ 0.15
# plot(x, y, line=(3, :dash, :lightblue), marker=(Shape(verts), 30, RGBA(0, 0, 0, 0.2)), bg=:pink, fg=:darkblue, xlim=(0, 1), ylim=(0, 1), leg=false)

In [ ]:
# using PlotlyJS

# function mesh_box(l,w,h,x,y,z)
#     PlotlyJS.mesh3d(
#         x = [x, x+l, x,   x+l, x,   x+l, x,   x+l],
#         y = [y, y,   y+w, y+w, y,   y,   y+w, y+w],
#         z = [z, z,   z,   z,   z+h, z+h, z+h, z+h],
#         i = [0, 0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7],
#         j = [3, 3, 5, 5, 6, 6, 3, 3, 5, 5, 6, 6],
#         k = [1, 2, 1, 4, 2, 4, 1, 2, 1, 4, 2, 4],
#         color = "rgb(200,100,100)",
#     )
# end

# PlotlyJS.plot(
#     mesh_box(1,1,1,0,0,0)
# )

In [16]:
function solve_gusto(model, N, MAX_ITERATIONS_NB, verbose = true)
    
    Xp,Up = initialize_trajectory(model, N)
    X, U  = copy(Xp)  , copy(Up) 

    SCPproblem = GuSTOProblem(model, N, Xp, Up)

    x_dim, u_dim = model.x_dim, model.u_dim

    Delta0, omega0, omegamax, 
        epsilon, rho0, rho1, 
        beta_succ, beta_fail, gamma_fail, 
        convergence_threshold = get_initial_gusto_parameters(model)

    Delta = Delta0
    omega = omega0

    X_all, U_all = [], []
    push!(X_all, copy(X))
    push!(U_all, copy(U))

    B_success = false
    it = 0

    while it<MAX_ITERATIONS_NB && 
            !(it!=0 && it!=1 && it!=2 && it!=3 && B_success && 
                convergence_metric(model,X,U,Xp,Up)<convergence_threshold) &&
            omega<omegamax
        if verbose
            println("-----------")
            println("Iteration $it")
            println("metric=$(convergence_metric(model,X,U,Xp,Up))")
            println("-----------")
        end

        Xp = copy(X)
        Up = copy(U)

        model.f, model.A, model.B = compute_dynamics(model, Xp, Up)

        reset_problem(SCPproblem, model)
        set_parameters(SCPproblem, model, Xp, Up, omega, Delta)
        define_cost(SCPproblem, model)
        define_constraints(SCPproblem, model)

        JuMP.optimize!(SCPproblem.solver_model)    
        X_sol = JuMP.value.(SCPproblem.X)
        U_sol = JuMP.value.(SCPproblem.U)

        # -----------
        # GuSTO Logic
        if it > 3
            if is_in_trust_region(model, X_sol, U_sol, Xp, Up, Delta)
                rho = accuracy_ratio(SCPproblem, model, X_sol, U_sol, Xp, Up)

                if rho > rho1
                    if verbose
                        println("Reject solution.")
                    end
                    Delta = beta_fail * Delta
                    omega     = omega
                    B_success = false

                else
                    if verbose
                        println("Accept solution.")
                    end
                    X = copy(X_sol)
                    U = copy(U_sol)
                    B_success = true
                    if rho < rho0
                        Delta = min(beta_succ*Delta, Delta0)
                    else
                        Delta = Delta
                    end
                    if satisfies_state_inequality_constraints(SCPproblem, model, X_sol, U_sol, Xp, Up, Delta)
                        omega = omega0
                    else
                        if verbose
                            println("Solution does not satisfy state constraints, increasing omega.")
                        end
                        omega = gamma_fail * omega
                        B_success = false
                    end
                end
            else
                if verbose
                    println("Reject solution (Outside trust region)")
                end
                Delta = Delta
                omega     = gamma_fail * omega
                B_success = false
            end
    #         if convergence_metric(model,X,U,Xp,Up) <0.6
    #             println("Convergence metric very small. Decreasing Delta.")
    #             Delta = Delta0./(2^it)
    #         end

        else # always accept first solution
            X = copy(X_sol)
            U = copy(U_sol)
        end


        # -----------

        push!(X_all, copy(X))
        push!(U_all, copy(U))


        it += 1

        diff_with_prev = norm(copy(X) - copy(Xp), Inf)
        if verbose
            println("x(k) - x(k-1) = $diff_with_prev")
            println("Parameters:")
            println("omega=$omega")
            println("delta=$Delta")
        end
    end
    curr_conv_metric = convergence_metric(model,X,U,Xp,Up)
    if (B_success && 
                curr_conv_metric<convergence_threshold)
        if verbose
            println("Converged")
        end
        isConverged = true
    else
        if verbose
            println("Not converged. Metric =$curr_conv_metric")
        end
        isConverged = false
    end
    return isConverged, X_all, U_all
end

solve_gusto (generic function with 2 methods)

In [17]:
# function plot_env_solution(model, X_all)
#     N = length(X_all)

#     idx = [1,2]
#     local fig
#     fig = plot(framestyle = :box)
#     for iter = 1:length(X_all)
#         X = X_all[iter]
#         if iter ==1 
#             color = :darkblue
#             plot!(fig, X[1,:], X[2,:], line=(5, :dash); c=color, lab = "Initial guess")
#         elseif iter == length(X_all)
#             color = :green
#             plot!(fig, X[1,:], X[2,:], line=(5, :dash), c=color, lab = "Final solution")
#         else
#             color = :blue
#             plot!(fig, X[1,:], X[2,:]; c=color, lab = "")
#         end
        
#     end

#     for obs_i = 1:length(model.obstacles)
#         p_obs, obs_radius = model.obstacles[obs_i][1], model.obstacles[obs_i][2]
#         plot_circle(p_obs[idx], obs_radius,lab =""; color=:red, fig=fig)
#     end
#     xlims!((model.x_min[1],model.x_max[1]))
#     ylims!((model.x_min[2],model.x_max[2]))
#     plot!(leg = true)
#     return fig
# end

In [18]:
function plot_env_and_trajs(model, X_all, labls = [""])
    N = length(X_all)

    idx = [1,2]
    local fig
    fig = plot(framestyle = :box)
    for iter = 1:length(X_all)
        X = X_all[iter]
        plot!(fig, X[1,:], X[2,:];line=(5, :dash), lab = labls[iter])
    end

    for obs_i = 1:length(model.obstacles)
        p_obs, obs_radius = model.obstacles[obs_i][1], model.obstacles[obs_i][2]
        plot_circle(p_obs[idx], obs_radius,lab =""; color=:red, fig=fig)
    end
    
    for obs_i = 1:length(model.poly_obstacles)
        plot_polyObs_corners_2d(fig, model.poly_obstacles[obs_i]; dims=idx)
    end
    
    xlims!((model.x_min[1],model.x_max[1]))
    ylims!((model.x_min[2],model.x_max[2]))
    plot!(leg = true)
    return fig
end

plot_env_and_trajs (generic function with 2 methods)

In [19]:
function plot_solution(model, X_all)
    N = length(X_all)

    idx = [2,3]
    local fig
    fig = plot()
    for iter = 1:length(X_all)
        X = X_all[iter]
        plot!(fig, X[1,:], X[2,:]; c=:blue)
    end

    for obs_i = 1:length(model.obstacles)
        p_obs, obs_radius = model.obstacles[obs_i][1], model.obstacles[obs_i][2]
        plot_circle(p_obs[idx], obs_radius; color=:red, fig=fig)
    end
    
    for obs_i = 1:length(model.poly_obstacles)
        plot_polyObs_corners_2d(fig, model.poly_obstacles[obs_i]; dims=idx)
    end
    
    

    return fig
end

plot_solution (generic function with 1 method)

# Simple Case

In [20]:
include("astrobee_se3_script.jl")

N = 50
MAX_ITERATIONS_NB = 100

model = AstrobeeSE3()

# Set up environment
model.x_max[1:3] = [10.0; 10.0; 1.0]
model.x_min[1:3] = [0.0; 0.0; 0.0]

obstacles = []
# obs = [[0.0,0.0,0.0], 6.0]
# push!(obstacles, obs)
# obs = [[5.0,5.0,0.0], 1.3]
# push!(obstacles, obs)
# obs = [[3.0,5.0,0.0], 1.3]
# push!(obstacles, obs)
# obs = [[5.0,3.0,0.0], 1.3]
# push!(obstacles, obs)
model.obstacles = obstacles

poly_obstacles = []
# obs = PolygonalObstacle([2.5,2.5,0], [5.0, 5.0, 2.0])
# push!(poly_obstacles, obs)
model.poly_obstacles = poly_obstacles

model.tf_guess = 110.
model.convergence_threshold = 0.5

model.x_init = [1.0;7.0;0.5;  0;0;0;  0.;0.;0.; 1.;  0;0;0]
model.x_final = [7.0 ;1.0;0.5;  0;0;0;  0.;0.;1.; 0.;  0;0;0]


@show(model.obstacles)
@show(model.poly_obstacles)

# plot_env_and_trajs(model, [], [])
isConverged, X_all, U_all = solve_gusto(model, N, MAX_ITERATIONS_NB, true)
# @show size(X_all)
# @show size(U_all)

model.obstacles = Any[]
model.poly_obstacles = Any[]
-----------
Iteration 0
metric=0.0
-----------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

x(k) - x(k-1) = 0.6231788894384609
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 1
metric=12.640211033869662
-----------
x(k) - x(k-1) = 0.03277069492008122
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 2
metric=0.49194428137438256
-----------
x(k) - x(k-1) = 0.015899258359204693
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 3
metric=0.2634712465362787
-----------
x(k) - x(k-1) = 0.007932025083340233
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 4
metric=0.1406480

(true, Any[[1.0 1.12245 … 6.87755 7.0; 7.0 6.87755 … 1.12245 1.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1.0 1.00536 … 6.98265 6.99902; 7.0 6.99464 … 1.01735 1.00098; … ; -8.0295e-35 -1.39694e-10 … 4.65108e-10 1.83478e-11; -8.52183e-35 -7.1806e-10 … 9.88625e-10 4.57346e-11], [1.0 1.00536 … 6.98266 6.99902; 7.0 6.99464 … 1.01734 1.00098; … ; -5.45413e-33 7.76645e-10 … 7.4926e-11 3.30294e-12; 0.0 1.16069e-9 … 1.19033e-9 5.03215e-11], [1.0 1.00536 … 6.98266 6.99902; 7.0 6.99464 … 1.01734 1.00098; … ; 0.0 1.13236e-9 … -1.00356e-9 -4.63952e-11; 1.30878e-33 1.05172e-9 … 5.01476e-10 2.21064e-11], [1.0 1.00536 … 6.98266 6.99902; 7.0 6.99464 … 1.01734 1.00098; … ; -3.3612e-34 -1.63073e-9 … -1.83993e-10 -8.11092e-12; -2.06815e-35 -4.58925e-10 … -7.51236e-10 -3.31165e-11], [1.0 1.00536 … 6.98265 6.99902; 7.0 6.99464 … 1.01735 1.00098; … ; -4.44889e-34 -1.44621e-9 … 5.08746e-10 2.2429e-11; 0.0 -1.41652e-9 … -2.29481e-9 -9.90061e-11]], Any[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0

In [23]:
X_soln = X_all[end]

13×50 Array{Float64,2}:
  1.0           1.00536       1.02304      …   6.98265       6.99902    
  7.0           6.99464       6.97696          1.01735       1.00098    
  0.5           0.5           0.5              0.5           0.5        
  0.0           0.00477711    0.0109713        0.00728992    0.00099962 
  0.0          -0.00477689   -0.0109709       -0.0072902    -0.00099962 
  0.0          -3.0511e-13   -7.21162e-13  …   6.96889e-12   6.93487e-12
  0.0           7.94986e-10   3.35645e-9       5.70801e-10  -2.75326e-13
  0.0          -8.11646e-10  -3.48468e-9      -2.58537e-9   -9.4922e-12 
 -2.30572e-29   0.00141222    0.0060673        1.00002       1.00004    
  1.0           0.999998      0.999976         0.00389949    5.7452e-6  
  6.47417e-30  -0.00251633   -0.0057783    …  -0.00346889   -0.000149577
 -4.44889e-34  -1.44621e-9   -3.31644e-9       5.08746e-10   2.2429e-11 
  0.0          -1.41652e-9   -3.14762e-9      -2.29481e-9   -9.90061e-11

In [27]:
# using BulletCollision

using Plots
# using RigidBodySim, RigidBodyDynamics
using MeshCat, MeshCatMechanisms
using CoordinateTransformations
using Interact, Reactive
# using MAT, FileIO, MeshIO
using MechanismGeometries
# using ForwardDiff
import GeometryTypes: HyperRectangle, HyperSphere, HomogenousMesh
# import ColorTypes: RGB, RGBA

# using StaticArrays, DataStructures
# using JuMP, Convex
# using Ipopt, Mosek, SCS
# using MosekTools
# using Gurobi
# using MathProgBase, MathOptInterface
# using NLsolve, DifferentialEquations

# using MAT
using GeometryTypes
using FillArrays
using LinearAlgebra

using AstrobeeRobot

N = 50
# Animate Astrobee trajectory
vis = Visualizer()
delete!(vis)

obstacles = []

push!(obstacles,
  HyperSphere(Point3f0([0.4,-0.10,0.]), Float32(0.1)))


# obstacles = []
# obs = [[0.4,-0.10,0.], 0.1]
# push!(obstacles, obs)

vis[:obstacle]
for (idx,ws) in enumerate(obstacles)
    setobject!(vis[:workspace][Symbol(string("ws",idx))],
        Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.26,0.26,0.3))))
end


# vis[:goal]
# for (idx,obs) in enumerate(env.keepout_zones)
#     setobject!(vis[:goal][:goal], 
#         Object(HyperSphere(Point3f0(x_goal[1:3]), 0.1f0),
#             MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))
# end

# vis[:workspace]
# for (idx,ws) in enumerate(env.keepin_zones)
#     if idx in (5,8)
#         setobject!(vis[:workspace][Symbol(string("ws",idx))],
#             Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3), depthWrite=false)))
#     else
#         setobject!(vis[:workspace][Symbol(string("ws",idx))],
#             Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3))))
#     end
# end

# vis[:obstacle]
# for (idx,ws) in enumerate(env.obstacle_set)
#     setobject!(vis[:workspace][Symbol(string("ws",idx+length(env.keepin_zones)))],
#         Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.26,0.26,0.3))))
# end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);

speed_factor = 3

Qs = Vector{Vector{Float64}}()
for k in 1:speed_factor:N
#     q = [quat_inv(TOS_SCP.SCPS.traj.X[7:10,k]); TOS_SCP.SCPS.traj.X[1:3,k]]
    q = [quat_inv(X_soln[7:10,k]); X_soln[1:3,k]]
    push!(Qs,q)
end

trans = Translation(14., -1., 7.)
rot = LinearMap(RotZ(-0.6)) ∘ LinearMap(RotY(-0.2))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.9)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

sleep(3)
setanimation!(mvis,1:length(Qs),Qs)

plot_in_cell = false
plot_in_cell ? IJuliaCell(vis) : open(vis)

┌ Info: Recompiling stale cache file /home/freeflyerhub/.julia/compiled/v1.1/Interact/XmYW4.ji for Interact [c601a237-2ae4-5e1e-952c-7a85b0c7eef1]
└ @ Base loading.jl:1184


instantiated a floating joint


┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8700
└ @ MeshCat /home/freeflyerhub/.julia/packages/MeshCat/J3ExE/src/servers.jl:24


Process(`xdg-open http://127.0.0.1:8700`, ProcessExited(0))

In [ ]:
plot_solution(model, X_all[end-1:end,:])

In [ ]:
include("astrobee_se3_script.jl")
x = [0;0.2;0.2]
obs = model.poly_obstacles[1]
plot_distances(x, obs)